In [92]:
import os
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import torchvision
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import numpy as np
from PIL import Image

In [43]:
# code for hpc:
# labels_path = '/groups/CS156b/data/student_labels/train2023.csv'
# img_dir = '/groups/CS156b/data/train'

# code for local:

# Training  
labels_path_train = 'data/train/labels/labels.csv'
img_dir_train = 'data/train/images'

# Test 
labels_path_test = 'data/test/ids.csv'
img_dir_test = 'data/test/images'

df_train = pd.read_csv(labels_path_train, delimiter='\t')[:-1]
display(df_train.head())

df_test = pd.read_csv(labels_path_test, delimiter='\t')[:-1]
display(df_test.head())

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Pneumonia,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,0,0,0,train/pid50512/study1/view1_frontal.jpg,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,1,1,1,train/pid21580/study2/view1_frontal.jpg,Female,87,Frontal,AP,-1.0,NaN,0.0,1.0,NaN,0.0,NaN,1.0,NaN
2,2,2,2,train/pid21580/study1/view1_frontal.jpg,Female,83,Frontal,AP,-1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN
3,3,3,3,train/pid21580/study1/view2_lateral.jpg,Female,83,Lateral,NaN,-1.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN
4,4,4,4,train/pid33839/study1/view1_frontal.jpg,Male,41,Frontal,AP,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Unnamed: 0,Id,Path
0,0,18,test/pid56785/study1/view1_frontal.jpg
1,1,19,test/pid56785/study1/view2_lateral.jpg
2,2,44,test/pid57943/study1/view1_frontal.jpg
3,3,45,test/pid57943/study2/view1_frontal.jpg
4,4,57,test/pid54805/study1/view1_frontal.jpg


In [83]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file, delimiter='\t')[:-1]
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        row = self.img_labels.iloc[idx]

        img_path = row['Path'].split('/')
        img_path = '/'.join(img_path[1:])
        img_path = os.path.join(self.img_dir, img_path)

        # image = read_image(img_path)
        image = Image.open(img_path) # PIL image for applying transform for pre-trained ResNet model 
        image = Image.fromarray(np.stack((image,)*3, axis=-1)) # convert to RGB
        label = list(row[-9:]) # extract label, the last 9 columns

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)

        return image, label

In [84]:
# Transform for ResNet 
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

training_data = CustomImageDataset(labels_path_train, img_dir_train, transform=transform)
test_data = CustomImageDataset(labels_path_test, img_dir_test, transform=transform)
a, b = training_data[0]
a, b

(tensor([[[-1.6727, -1.6555, -1.5870,  ..., -0.9192, -0.9534, -1.0048],
          [-1.6384, -1.5870, -1.5014,  ..., -1.1075, -1.1589, -1.2617],
          [-1.6042, -1.5528, -1.3644,  ..., -1.2103, -1.2274, -1.2617],
          ...,
          [ 1.3927,  1.4783,  1.4954,  ...,  1.7694,  1.7523,  1.7523],
          [ 1.4440,  1.5297,  1.5468,  ...,  1.7694,  1.8379,  1.8208],
          [ 1.4954,  1.5468,  1.5810,  ...,  1.8550,  1.7865,  1.8379]],
 
         [[-1.5805, -1.5630, -1.4930,  ..., -0.8102, -0.8452, -0.8978],
          [-1.5455, -1.4930, -1.4055,  ..., -1.0028, -1.0553, -1.1604],
          [-1.5105, -1.4580, -1.2654,  ..., -1.1078, -1.1253, -1.1604],
          ...,
          [ 1.5532,  1.6408,  1.6583,  ...,  1.9384,  1.9209,  1.9209],
          [ 1.6057,  1.6933,  1.7108,  ...,  1.9384,  2.0084,  1.9909],
          [ 1.6583,  1.7108,  1.7458,  ...,  2.0259,  1.9559,  2.0084]],
 
         [[-1.3513, -1.3339, -1.2641,  ..., -0.5844, -0.6193, -0.6715],
          [-1.3164, -1.2641,

In [85]:
train_dataloader = DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [86]:
classes = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Pneumonia', 'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']

In [93]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)

if torch.cuda.is_available():
    model.to_cuda()
model.eval()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=5)

Using cache found in /Users/oliviaxu/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/oliviaxu/anaconda3/envs/myenv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/oliviaxu/anaconda3/envs/myenv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
n_epochs = 200

for epoch in range(n_epochs):
    losses = []
    running_loss = 0
    for i, inp in enumerate(train_dataloader):
        inputs, labels = inp
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()
    
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i%100 == 0 and i > 0:
            print(f'Loss [{epoch+1}, {i}](epoch, minibatch): ', running_loss / 100)
            running_loss = 0.0

    avg_loss = sum(losses)/len(losses)
    scheduler.step(avg_loss)
            
print('Training Done')